In [26]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import plotly.express as px
import plotly.graph_objects as go
pd.options.display.float_format = '{:.0f}'.format

In [2]:
df = pd.read_csv("nst-est2019-alldata.csv")
df.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,10,0,0,0,United States,308745538,308758105,309321666,311556874,313830990,...,0,2,3,3,3,3,3,3,2,2
1,20,1,0,0,Northeast Region,55317240,55318443,55380134,55604223,55775216,...,-5,1,-0,-0,-1,-2,-2,-2,-2,-3
2,20,2,0,0,Midwest Region,66927001,66929725,66974416,67157800,67336743,...,-2,-1,-1,-0,-1,-1,-1,-1,-1,-1
3,20,3,0,0,South Region,114555744,114563030,114866680,116006522,117241208,...,3,5,6,5,6,7,7,6,5,5
4,20,4,0,0,West Region,71945553,71946907,72100436,72788329,73477823,...,1,3,3,3,4,5,5,4,3,2


In [3]:
df = df.drop(df.columns[27:], axis = 1)
df = df.drop(df.columns[0], axis = 1)

In [4]:
df_house = df[['NAME','CENSUS2010POP','POPESTIMATE2010','POPESTIMATE2019']]
df_house = df_house.drop([0,1,2,3,4,13,56])
df_house.reset_index(drop=True, inplace=True)
#df_house.head()

In [5]:
df_house['POPESTIMATE2020'] = np.zeros(50)
df_house['CENSUS2020POP'] = np.zeros(50)

for index, row in df_house.iterrows():
    rate = (row['POPESTIMATE2019']/row['POPESTIMATE2010']) - 1
    rate = rate/9
    rate_census = rate*9/12
    
    pop_2020_est = row['POPESTIMATE2019']*np.exp(rate)
    pop_2020_census = row['POPESTIMATE2019']*np.exp(rate_census)
    
    df_house.loc[index,'POPESTIMATE2020'] = pop_2020_est
    df_house.loc[index,'CENSUS2020POP'] = pop_2020_census
    
df_house.head()
    

,NAME,CENSUS2010POP,POPESTIMATE2010,POPESTIMATE2019,POPESTIMATE2020,CENSUS2020POP
0,Alabama,4779736,4785437,4903185,4916608,4913249
1,Alaska,710231,713910,731545,733556,733052
2,Arizona,6392017,6407172,7278717,7389563,7361695
3,Arkansas,2915918,2921964,3017804,3028822,3026064
4,California,37253956,37319502,39512223,39771017,39706160


In [6]:
seats_2010 = [1 for i in range(50)]
seats_2020 = [1 for i in range(50)]
priority_2010 = [0 for i in range(50)]
priority_2020 = [0 for i in range(50)]

seats_left = 435-50

for index in range(50):
    num_seats_2020 = seats_2020[index]
    priority_2020[index] = df_house.loc[index,"CENSUS2020POP"]/math.sqrt(num_seats_2020*(num_seats_2020+1))
    
    num_seats_2010 = seats_2010[index]
    priority_2010[index] = df_house.loc[index,"CENSUS2010POP"]/math.sqrt(num_seats_2010*(num_seats_2010+1))

In [7]:
def allocate(df,seats,priority,year):
    max_index = priority.index(max(priority))
    seats[max_index] += 1
    
    if (year == 2010):
        new_priority = df.loc[max_index,"CENSUS2010POP"]/math.sqrt(seats[max_index]*(seats[max_index]+1))
    else:
        new_priority = df.loc[max_index,"CENSUS2020POP"]/math.sqrt(seats[max_index]*(seats[max_index]+1))
    priority[max_index] = new_priority

In [8]:
for year in range(seats_left):
    allocate(df_house,seats_2010,priority_2010,2010)
    allocate(df_house,seats_2020,priority_2020,2020)
seats_2010 = [x+2 for x in seats_2010]
seats_2020 = [x+2 for x in seats_2020]
seats_diff = [x - y for x,y in zip(seats_2020,seats_2010)]

In [9]:
df_ec = df_house['NAME'].to_frame()
df_ec['Seats2010'] = pd.Series(seats_2010)
df_ec['Seats2020'] = pd.Series(seats_2020)
df_ec['SeatsChange'] = pd.Series(seats_diff)
df_ec.head()

,NAME,Seats2010,Seats2020,SeatsChange
0,Alabama,9,8,-1
1,Alaska,3,3,0
2,Arizona,11,12,1
3,Arkansas,6,6,0
4,California,55,54,-1


In [34]:
state_codes = pd.read_csv("StateCode.csv")
df_elec_col = pd.read_csv("Electoral_College.csv")
state_codes = state_codes.drop(index = [2,9,10,13,23,39,43,45,53]).reset_index(drop=True)
df_ec['Code'] = state_codes['Code']

In [42]:
plot = go.Figure(data = go.Choropleth(locations = df_ec['Code'], z = df_ec['Seats2010'], locationmode = 'USA-states', 
            colorbar_title = "EC Votes", colorscale = "Reds", text = df_ec['NAME'].astype(str)))
plot.update_layout(title_text = 'Electoral Votes by State After 2010 Census', geo_scope='usa')
plot.show()

In [41]:
plot = go.Figure(data = go.Choropleth(locations = df_ec['Code'], z = df_ec['Seats2020'], locationmode = 'USA-states', 
            colorbar_title = "EC Votes", colorscale = "Reds", text = df_ec['NAME'].astype(str)))
plot.update_layout(title_text = 'Predicted Electoral Votes by State After 2020 Census', geo_scope='usa')
plot.show()

In [43]:
plot = go.Figure(data = go.Choropleth(locations = df_ec['Code'], z = df_ec['SeatsChange'], locationmode = 'USA-states', 
            colorbar_title = "Change", colorscale = "RdBu", text = df_ec['NAME'].astype(str)))
plot.update_layout(title_text = 'Predicted Change in Electoral Votes After 2020 Census', geo_scope='usa')
plot.show()